# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Set the path to citis.csv
cities_path = "../output_data/cities.csv"

# Read the cities.csv
cities_df = pd.read_csv(cities_path)

# # Set the index to the City ID
cities_df = cities_df.set_index('City ID')

# output cities_df
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Kapaa,75,US,1603048659,94,22.08,-159.32,80.01,13.87
1,Esim,29,GH,1603048501,88,4.87,-2.24,77.43,7.05
2,Antalya Province,40,TR,1603048483,83,36.77,30.72,73.40,3.36
3,Koło,71,PL,1603048728,97,52.20,18.64,46.00,1.01
4,Loreto,0,MX,1603048728,25,22.27,-101.97,76.08,8.23
...,...,...,...,...,...,...,...,...,...
520,Yekaterinogradskaya,1,RU,1603049369,62,43.76,44.23,59.00,4.47
521,Dwārka,54,IN,1603049173,81,22.24,68.97,83.82,10.63
522,Stepnyak,0,KZ,1603049369,91,52.83,70.78,31.44,5.19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
weights = cities_df['Humidity'].astype(float)

cities_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

cities_fig.add_layer(heat_layer)

cities_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Drop any null values
cities_df = cities_df.dropna()


# Setup variables for searching for the perfect vacation spot
low_temp = 70
high_temp = 80
max_wind_speed = 10
cloudiness = 0
humidity = 30

# Create a DataFrame with only cities that have our ideal weather
vacation_cities_df = cities_df.loc[cities_df['Wind Speed'] < max_wind_speed]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Cloudiness'] ==  cloudiness]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] > low_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Max Temp'] < high_temp]
vacation_cities_df = vacation_cities_df.loc[vacation_cities_df['Humidity'] < humidity]
vacation_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
4,Loreto,0,MX,1603048728,25,22.27,-101.97,76.08,8.23
33,Constitucion,0,MX,1603048730,24,23.99,-104.67,78.80,9.91
115,Dera Bugti,0,PK,1603048856,18,29.03,69.15,71.51,2.91
194,Shakawe,0,BW,1603048923,17,-18.37,21.85,75.31,4.07
198,Riyadh,0,SA,1603048791,23,24.69,46.72,73.40,6.82
334,Viedma,0,AR,1603049115,19,-40.81,-63.00,78.01,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create a new dataframe and add a Hotel Name column
hotel_df = pd.DataFrame(vacation_cities_df,
                      columns=['City','Country','Lat','Lng','Hotel Name'])

hotel_df

,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
4,Loreto,MX,22.27,-101.97,NaN
33,Constitucion,MX,23.99,-104.67,NaN
115,Dera Bugti,PK,29.03,69.15,NaN
194,Shakawe,BW,-18.37,21.85,NaN
198,Riyadh,SA,24.69,46.72,NaN
334,Viedma,AR,-40.81,-63.00,NaN


In [6]:
# find the closest hotel for all of our possible vacation cities

# Define the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through possible vacation cities
for index, row in hotel_df.iterrows():
    
    # Set a variable for the latitude and Longitude of our city
    city_lat_lng = f"{row['Lat']},{row['Lng']}"
    
    # Define the api call parameters
    params = {
               "location": city_lat_lng,  
               "rankby": "distance",
               "types": "lodging",
               "key": g_key
             }
    # Output that we are retrieving information for our city
    print(f"Retrieving Results for Index {index}: {row['City']}, {row['Country']}.")
   
    # assemble url and make API request
    response_json = requests.get(base_url, params=params).json()
    
    # extract results
    results = response_json['results']

    # If we got a hotel add it to the DataFrame
    # if we don't get a hotel then output a message that we are skipping it
    try:
        # Output the closet hotel
        print(f"Closest hotel is {results[0]['name']}.")
        
        # Update the data frame with the hotel name
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']
 
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------------------------------------------")

hotel_df   

Retrieving Results for Index 4: Loreto, MX.
Closest hotel is José de Jesús Velásquez mora.
-----------------------------------------------------
Retrieving Results for Index 33: Constitucion, MX.
Closest hotel is Mi Cantón.
-----------------------------------------------------
Retrieving Results for Index 115: Dera Bugti, PK.
Closest hotel is SARDAR GHULAM MUSTAFA KHAN BUGTI.
-----------------------------------------------------
Retrieving Results for Index 194: Shakawe, BW.
Closest hotel is Ditlhapi Guest House.
-----------------------------------------------------
Retrieving Results for Index 198: Riyadh, SA.
Closest hotel is عمارة الشلهوب.
-----------------------------------------------------
Retrieving Results for Index 334: Viedma, AR.
Closest hotel is Alta Patagonia Apart.
-----------------------------------------------------


,City,Country,Lat,Lng,Hotel Name
City ID,,,,,
4,Loreto,MX,22.27,-101.97,José de Jesús Velásquez mora
33,Constitucion,MX,23.99,-104.67,Mi Cantón
115,Dera Bugti,PK,29.03,69.15,SARDAR GHULAM MUSTAFA KHAN BUGTI
194,Shakawe,BW,-18.37,21.85,Ditlhapi Guest House
198,Riyadh,SA,24.69,46.72,عمارة الشلهوب
334,Viedma,AR,-40.81,-63.00,Alta Patagonia Apart


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
cities_fig.add_layer(markers)

# Display Map
cities_fig

Figure(layout=FigureLayout(height='420px'))